In [ ]:
input_file = "drawing.svg"
# input_file = "transform.svg"

In [ ]:
from svgpathtools import Document, Path, Line, QuadraticBezier, CubicBezier, Arc, is_bezier_path, svg2paths, wsvg

doc = Document(input_file)
paths = doc.paths()

In [ ]:
def lerp(a,b,t):
    return (1-t)*a+t*b

def line_to_cubic(line : Line):
    p_0,p_3 = line.bpoints()
    return CubicBezier(p_0, lerp(p_0, p_3, 1/3), lerp(p_0, p_3, 2/3), p_3)

def quadratic_to_cubic(quad : QuadraticBezier):
    q_0,q_1,q_2 = quad.bpoints()
    return CubicBezier(q_0, lerp(q_0,q_1, 2/3), lerp(q_1,q_2, 1/3), q_2)

def segment_as_cubic(seg):
    if isinstance(seg,Line):
        return line_to_cubic(seg)
    elif isinstance(seg,QuadraticBezier):
        return quadratic_to_cubic(seg)
    elif isinstance(seg, CubicBezier):
        return seg
    elif isinstance(seg,Arc):
        raise Exception("'Arc' type not yet supported")
    else:
        raise Exception(f"'{type(seg)}' type not supported yet")


In [ ]:
for p_idx, p in enumerate(paths):
    for seg_idx, seg in enumerate(p):
        try:
            cubic =  segment_as_cubic(seg)
            print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}\n\tas cubic: {cubic}")
        except Exception as err:
            print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}")
            print(f"parsed unsupported type {type(seg)}. Msg={err}")


In [ ]:
import numpy as np

# Create an mfem mesh

header = """
MFEM mesh v1.0

# MFEM Geometry Types (see mesh/geom.hpp):
#
# POINT       = 0
# SEGMENT     = 1
# TRIANGLE    = 2
# SQUARE      = 3
# TETRAHEDRON = 4
# CUBE        = 5
#

dimension
1
"""

vert_cnt = 0
elem_cnt = 0
verts = []
dofs = []
elts = []

print(paths)

for p_idx, p in enumerate(paths):

    if not is_bezier_path(p):
        continue

    for seg_idx, seg in enumerate(p):
        cubic = segment_as_cubic(seg)
        elts.append(" ".join(map(str,[p_idx + 1, 1, vert_cnt, vert_cnt + 1])))
        verts.append(" ".join(map(str,[cubic.start.real, cubic.start.imag])))
        verts.append(" ".join(map(str,[cubic.end.real, cubic.end.imag])))
        dofs.append(" ".join(map(str,[cubic.control1.real, cubic.control1.imag])))
        dofs.append(" ".join(map(str,[cubic.control2.real, cubic.control2.imag])))
        vert_cnt += 2
        elem_cnt += 1 

mfem_file = []
mfem_file.append(header)
mfem_file.append(f"""
elements
{elem_cnt}
{"\n".join(elts)}
""")

mfem_file.append(f"""
boundary
0
""")

mfem_file.append(f"""
vertices
{vert_cnt}
""")

mfem_file.append(f"""
nodes
FiniteElementSpace
FiniteElementCollection: H1Pos_1D_P3
VDim: 2
Ordering: 1

{"\n".join(verts)}
{"\n".join(dofs)}
""")

fname = "drawing.mesh"
with open(fname, mode='w') as f:
    f.write("\n".join(mfem_file))
    print(f"wrote '{fname}' with {vert_cnt} vertices and {elem_cnt} elements")
